In [ ]:
!pip3 install scikit-learn==0.19.1

     |████████████████████████████████| 12.4MB 365kB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/My Drive/ML_UseCases/Kidney_Disease/Data.csv')

In [ ]:
df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,48,80,1.02,1,0,121,36,1.2,?,?,15.4,44,7800,5.2,?,normal,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,?,18,0.8,?,?,11.3,38,6000,?,?,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,423,53,1.8,?,?,9.6,31,7500,?,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,117,56,3.8,111,2.5,11.2,32,6700,3.9,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,106,26,1.4,?,?,11.6,35,7300,4.6,normal,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.02,0,0,140,49,0.5,150,4.9,15.7,47,6700,4.9,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
396,42,70,1.025,0,0,75,31,1.2,141,3.5,16.5,54,7800,6.2,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
397,12,80,1.02,0,0,100,26,0.6,137,4.4,15.8,49,6600,5.4,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
398,17,60,1.025,0,0,114,50,1,135,4.9,14.2,51,7200,5.9,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd


In [ ]:
df=df.replace('?',np.nan)
df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,48,80,1.02,1,0,121,36,1.2,NaN,NaN,15.4,44,7800,5.2,NaN,normal,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,NaN,18,0.8,NaN,NaN,11.3,38,6000,NaN,NaN,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,423,53,1.8,NaN,NaN,9.6,31,7500,NaN,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,117,56,3.8,111,2.5,11.2,32,6700,3.9,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,106,26,1.4,NaN,NaN,11.6,35,7300,4.6,normal,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.02,0,0,140,49,0.5,150,4.9,15.7,47,6700,4.9,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
396,42,70,1.025,0,0,75,31,1.2,141,3.5,16.5,54,7800,6.2,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
397,12,80,1.02,0,0,100,26,0.6,137,4.4,15.8,49,6600,5.4,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
398,17,60,1.025,0,0,114,50,1,135,4.9,14.2,51,7200,5.9,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd


In [ ]:
df.isna().sum()

age        9
bp        12
sg        47
al        46
su        49
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
rbc      152
pc        65
pcc        4
ba         4
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [ ]:
#df=df.dropna(subset=['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane','class'])
X =df.iloc[:, :-1].values
Y = df.iloc[:, 24].values
 #missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(X[:, 0:14])
X[:,0:14] = imputer.transform(X[:,0:14])
x=pd.DataFrame(X)
y=pd.DataFrame(Y)
x=x.replace(np.nan,'nun')
y=y.replace(np.nan,'nun')
X=np.array(x)
Y=np.array(y)

In [ ]:
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
for i in range(14,24):
  X[:, i] = labelencoder_X.fit_transform(X[:, i])
x=pd.DataFrame(X)

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
Y = labelencoder_y.fit_transform(Y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score  
classifier = SVC(kernel = 'sigmoid', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test,y_pred)

conf = confusion_matrix(y_test,y_pred)

print(f"Training accuracy: {acc_train}")
print(f"Test accuracy: {acc_test}")
print(f"Confusion matrix:\n{conf}")

Training accuracy: 0.9928571428571429
Test accuracy: 0.9916666666666667
Confusion matrix:
[[71  1]
 [ 0 48]]
